In [1]:
import pycaret
from pycaret.regression import *
from pycaret.classification import *
import numpy as np
import pandas as pd
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
import scipy.stats as stats
from tableone import TableOne

In [2]:
# baseline
# pre = pd.read_csv (r"C:\Users\user\Desktop\vsc\Severance_Hospital\12.19_정현도_alpha_feature\FDATA_Pre_2B_GAUS_NORM_RESAMPLE000_5Sig_2023-12-17-19-39-33_radiomics_features_ALL.csv" , index_col=0)

# first
# pre = pd.read_csv (r"C:\Users\user\Desktop\vsc\Severance_Hospital\12.19_정현도_alpha_feature\CLAHE_pre.csv" , index_col=0)

# second
# pre = pd.read_csv (r"C:\Users\user\Desktop\vsc\Severance_Hospital\12.19_정현도_alpha_feature\HIST_PRE.csv" , index_col=0)

# third
pre = pd.read_csv (r"C:\Users\user\Desktop\vsc\Severance_Hospital\12.19_정현도_alpha_feature\ZNORM_PRE.csv" , index_col=0)


pre.set_index('pt_no', inplace=True)
current_pre = pre.columns.tolist()
new_pre = {col: 'pre_' + col for col in current_pre}
pre = pre.rename(columns=new_pre)
pre

,pre_exponential_firstorder_90Percentile,pre_exponential_firstorder_Energy,pre_exponential_firstorder_Entropy,pre_exponential_firstorder_InterquartileRange,pre_exponential_firstorder_Kurtosis,pre_exponential_firstorder_Maximum,pre_exponential_firstorder_Mean,pre_exponential_firstorder_MeanAbsoluteDeviation,pre_exponential_firstorder_Median,pre_exponential_firstorder_Minimum,...,pre_wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,pre_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,pre_wavelet-LLL_glszm_ZoneEntropy,pre_wavelet-LLL_glszm_ZonePercentage,pre_wavelet-LLL_glszm_ZoneVariance,pre_wavelet-LLL_ngtdm_Busyness,pre_wavelet-LLL_ngtdm_Coarseness,pre_wavelet-LLL_ngtdm_Complexity,pre_wavelet-LLL_ngtdm_Contrast,pre_wavelet-LLL_ngtdm_Strength
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,1.488204,33747.767654,-3.203427e-16,0.029460,16.777797,1.663273,1.438424,0.050884,1.453668,0.849846,...,2.612107,0.010728,2.807355,0.000432,2.843862e+07,9.302526,0.006286,0.172151,0.000534,0.029250
48418610,1.934302,19441.725057,4.527199e-02,0.074769,23.369120,2.045145,1.857942,0.054222,1.867810,1.070415,...,0.300325,0.020510,3.095795,0.001957,2.514198e+06,4.187711,0.020254,0.069522,0.000129,0.093145
38581294,2.838803,78431.581990,6.626930e-02,0.079135,51.577812,3.572165,2.755322,0.071705,2.773258,0.989683,...,4.913526,0.155149,4.271371,0.003204,2.847141e+06,3.673864,0.005602,0.667983,0.000255,0.144014
47730588,1.913005,21963.054677,5.460113e-02,0.047842,21.389449,2.189460,1.844602,0.068655,1.878945,0.932321,...,7.473821,0.036400,3.559080,0.003893,1.294095e+06,8.059244,0.004167,0.933599,0.001423,0.053240
49696709,2.981339,81806.067277,1.434771e-02,0.111454,29.867949,3.472855,2.858551,0.082533,2.870545,0.975520,...,5.434147,0.065477,4.001823,0.002302,3.862245e+06,2.817298,0.007240,0.583065,0.000092,0.189203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56124435,2.397334,79676.170423,2.077586e-01,0.094314,21.825817,2.655513,2.288176,0.080719,2.314499,1.040494,...,4.800308,0.060496,4.704448,0.002901,3.836082e+06,17.611352,0.001549,1.395922,0.001495,0.039096
50438996,2.471583,16799.993618,2.513645e-01,0.118093,22.979982,2.718759,2.339055,0.110733,2.382084,0.961707,...,2.776515,0.057959,3.456565,0.004921,2.752990e+05,8.664636,0.003766,2.898350,0.008258,0.079879
54990434,2.204391,12391.249792,3.038270e-01,0.060698,32.538823,2.496660,2.133809,0.058552,2.156958,0.992981,...,2.060174,0.107283,2.977217,0.005895,3.633319e+05,3.167530,0.010460,0.948817,0.001024,0.123691


In [3]:
# ypCR_data = pd.read_excel("/media/yonsei/Mike/clinical_immunologic.xlsx", index_col=0) 
ypCR_data = pd.read_excel (r"C:\Users\user\Desktop\vsc\Severance_Hospital\12.19_정현도_alpha_feature\pre\clinical_immunologic.xlsx" , index_col=0)
ypCR_data.set_index('pt_no', inplace=True)
ypCR_data = ypCR_data[ypCR_data['Pathol_available'] == 1]
ypCR_data

,name,Ineligible,Ineligible_2019,Pathol_available,Hospital,note,base.fu,date.dx,dob,age,...,note.1,other.malig,pre_PDL1_CPS,pre_PDL1_IC,pre_CD3_num_Pos_mm,pre_CD8_num_Pos_mm,post_PDL1_CPS,post_PDL1_IC,post_CD3_num_Pos_mm,post_CD8_num_Pos_mm
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,정종일,0,0,1.0,1,NaN,2016-11-17,2016-10-24,1946-03-05,69.731507,...,NaN,NaN,5.0,5.0,366.0532,51.9522,10.0,10.0,3171.7243,890.4297
17815426,김진영,1,0,1.0,1,NaN,2021-07-30,2021-07-08,1955-01-04,65.660274,...,NaN,NaN,50.0,40.0,637.6734,374.4912,80.0,80.0,1532.5386,1298.4060
20109646,서대현,0,0,1.0,1,NaN,2018-07-09,2018-05-18,1951-06-20,66.134247,...,NaN,NaN,0.5,0.5,60.6479,39.7833,0.0,0.0,100.5308,43.6339
22320836,최흥우,0,0,1.0,1,NaN,2016-06-07,2016-05-18,1947-07-11,67.961644,...,"diaphragmatic hernia reduction (2020/3/17, 201...",NaN,10.0,10.0,608.9312,140.8317,10.0,10.0,1176.5166,1516.6479
32221501,박남훈,0,0,1.0,1,NaN,2010-10-11,2010-09-17,1939-10-16,70.013699,...,NaN,NaN,1.0,1.0,504.0535,64.6015,10.0,10.0,1245.4099,1155.8683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55941790,최정옥,1,0,1.0,1,Previous malignancy: Breast ca (하지만 1995년이라 ok),2021-10-26,2021-09-29,1961-11-27,59.093151,...,NaN,NaN,10.0,5.0,1152.7963,471.6934,100.0,70.0,1477.2669,868.1072
56124435,김정자,1,0,1.0,1,NaN,2021-12-03,2021-11-09,1963-01-13,58.082192,...,NaN,NaN,0.0,0.0,145.0536,31.3657,10.0,10.0,479.3238,389.0902
56131318,홍성배,1,0,1.0,1,NaN,2021-11-23,2021-11-10,1951-05-20,69.542466,...,NaN,NaN,10.0,10.0,5283.4120,5773.6300,50.0,50.0,4583.2514,4448.2354


In [4]:
pre['post_PDL1_CPS'] = ypCR_data['post_PDL1_CPS']
pre

,pre_exponential_firstorder_90Percentile,pre_exponential_firstorder_Energy,pre_exponential_firstorder_Entropy,pre_exponential_firstorder_InterquartileRange,pre_exponential_firstorder_Kurtosis,pre_exponential_firstorder_Maximum,pre_exponential_firstorder_Mean,pre_exponential_firstorder_MeanAbsoluteDeviation,pre_exponential_firstorder_Median,pre_exponential_firstorder_Minimum,...,pre_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,pre_wavelet-LLL_glszm_ZoneEntropy,pre_wavelet-LLL_glszm_ZonePercentage,pre_wavelet-LLL_glszm_ZoneVariance,pre_wavelet-LLL_ngtdm_Busyness,pre_wavelet-LLL_ngtdm_Coarseness,pre_wavelet-LLL_ngtdm_Complexity,pre_wavelet-LLL_ngtdm_Contrast,pre_wavelet-LLL_ngtdm_Strength,post_PDL1_CPS
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,1.488204,33747.767654,-3.203427e-16,0.029460,16.777797,1.663273,1.438424,0.050884,1.453668,0.849846,...,0.010728,2.807355,0.000432,2.843862e+07,9.302526,0.006286,0.172151,0.000534,0.029250,10.0
48418610,1.934302,19441.725057,4.527199e-02,0.074769,23.369120,2.045145,1.857942,0.054222,1.867810,1.070415,...,0.020510,3.095795,0.001957,2.514198e+06,4.187711,0.020254,0.069522,0.000129,0.093145,1.0
38581294,2.838803,78431.581990,6.626930e-02,0.079135,51.577812,3.572165,2.755322,0.071705,2.773258,0.989683,...,0.155149,4.271371,0.003204,2.847141e+06,3.673864,0.005602,0.667983,0.000255,0.144014,1.0
47730588,1.913005,21963.054677,5.460113e-02,0.047842,21.389449,2.189460,1.844602,0.068655,1.878945,0.932321,...,0.036400,3.559080,0.003893,1.294095e+06,8.059244,0.004167,0.933599,0.001423,0.053240,12.0
49696709,2.981339,81806.067277,1.434771e-02,0.111454,29.867949,3.472855,2.858551,0.082533,2.870545,0.975520,...,0.065477,4.001823,0.002302,3.862245e+06,2.817298,0.007240,0.583065,0.000092,0.189203,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56124435,2.397334,79676.170423,2.077586e-01,0.094314,21.825817,2.655513,2.288176,0.080719,2.314499,1.040494,...,0.060496,4.704448,0.002901,3.836082e+06,17.611352,0.001549,1.395922,0.001495,0.039096,10.0
50438996,2.471583,16799.993618,2.513645e-01,0.118093,22.979982,2.718759,2.339055,0.110733,2.382084,0.961707,...,0.057959,3.456565,0.004921,2.752990e+05,8.664636,0.003766,2.898350,0.008258,0.079879,0.0
54990434,2.204391,12391.249792,3.038270e-01,0.060698,32.538823,2.496660,2.133809,0.058552,2.156958,0.992981,...,0.107283,2.977217,0.005895,3.633319e+05,3.167530,0.010460,0.948817,0.001024,0.123691,30.0


In [5]:
# bin_labels = [0, 1, 2]
# bins = [-1, 1, 10, float('inf')]
# pre['encoded_PDL1'] = pd.cut(pre['post_PDL1_CPS'], bins=bins, labels=bin_labels)
# pre
conditions = [
    (pre['post_PDL1_CPS'] >= 0) & (pre['post_PDL1_CPS'] < 1),
    (pre['post_PDL1_CPS'] >= 1) & (pre['post_PDL1_CPS'] <= 10),
    (pre['post_PDL1_CPS'] > 10)
]
choices = [0, 1, 2]
pre['encoded_PDL1'] = np.select(conditions, choices, default=np.nan)
pre

,pre_exponential_firstorder_90Percentile,pre_exponential_firstorder_Energy,pre_exponential_firstorder_Entropy,pre_exponential_firstorder_InterquartileRange,pre_exponential_firstorder_Kurtosis,pre_exponential_firstorder_Maximum,pre_exponential_firstorder_Mean,pre_exponential_firstorder_MeanAbsoluteDeviation,pre_exponential_firstorder_Median,pre_exponential_firstorder_Minimum,...,pre_wavelet-LLL_glszm_ZoneEntropy,pre_wavelet-LLL_glszm_ZonePercentage,pre_wavelet-LLL_glszm_ZoneVariance,pre_wavelet-LLL_ngtdm_Busyness,pre_wavelet-LLL_ngtdm_Coarseness,pre_wavelet-LLL_ngtdm_Complexity,pre_wavelet-LLL_ngtdm_Contrast,pre_wavelet-LLL_ngtdm_Strength,post_PDL1_CPS,encoded_PDL1
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,1.488204,33747.767654,-3.203427e-16,0.029460,16.777797,1.663273,1.438424,0.050884,1.453668,0.849846,...,2.807355,0.000432,2.843862e+07,9.302526,0.006286,0.172151,0.000534,0.029250,10.0,1.0
48418610,1.934302,19441.725057,4.527199e-02,0.074769,23.369120,2.045145,1.857942,0.054222,1.867810,1.070415,...,3.095795,0.001957,2.514198e+06,4.187711,0.020254,0.069522,0.000129,0.093145,1.0,1.0
38581294,2.838803,78431.581990,6.626930e-02,0.079135,51.577812,3.572165,2.755322,0.071705,2.773258,0.989683,...,4.271371,0.003204,2.847141e+06,3.673864,0.005602,0.667983,0.000255,0.144014,1.0,1.0
47730588,1.913005,21963.054677,5.460113e-02,0.047842,21.389449,2.189460,1.844602,0.068655,1.878945,0.932321,...,3.559080,0.003893,1.294095e+06,8.059244,0.004167,0.933599,0.001423,0.053240,12.0,2.0
49696709,2.981339,81806.067277,1.434771e-02,0.111454,29.867949,3.472855,2.858551,0.082533,2.870545,0.975520,...,4.001823,0.002302,3.862245e+06,2.817298,0.007240,0.583065,0.000092,0.189203,5.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56124435,2.397334,79676.170423,2.077586e-01,0.094314,21.825817,2.655513,2.288176,0.080719,2.314499,1.040494,...,4.704448,0.002901,3.836082e+06,17.611352,0.001549,1.395922,0.001495,0.039096,10.0,1.0
50438996,2.471583,16799.993618,2.513645e-01,0.118093,22.979982,2.718759,2.339055,0.110733,2.382084,0.961707,...,3.456565,0.004921,2.752990e+05,8.664636,0.003766,2.898350,0.008258,0.079879,0.0,0.0
54990434,2.204391,12391.249792,3.038270e-01,0.060698,32.538823,2.496660,2.133809,0.058552,2.156958,0.992981,...,2.977217,0.005895,3.633319e+05,3.167530,0.010460,0.948817,0.001024,0.123691,30.0,2.0


In [6]:
pre.drop(columns=['post_PDL1_CPS'], inplace=True)
pre

,pre_exponential_firstorder_90Percentile,pre_exponential_firstorder_Energy,pre_exponential_firstorder_Entropy,pre_exponential_firstorder_InterquartileRange,pre_exponential_firstorder_Kurtosis,pre_exponential_firstorder_Maximum,pre_exponential_firstorder_Mean,pre_exponential_firstorder_MeanAbsoluteDeviation,pre_exponential_firstorder_Median,pre_exponential_firstorder_Minimum,...,pre_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,pre_wavelet-LLL_glszm_ZoneEntropy,pre_wavelet-LLL_glszm_ZonePercentage,pre_wavelet-LLL_glszm_ZoneVariance,pre_wavelet-LLL_ngtdm_Busyness,pre_wavelet-LLL_ngtdm_Coarseness,pre_wavelet-LLL_ngtdm_Complexity,pre_wavelet-LLL_ngtdm_Contrast,pre_wavelet-LLL_ngtdm_Strength,encoded_PDL1
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,1.488204,33747.767654,-3.203427e-16,0.029460,16.777797,1.663273,1.438424,0.050884,1.453668,0.849846,...,0.010728,2.807355,0.000432,2.843862e+07,9.302526,0.006286,0.172151,0.000534,0.029250,1.0
48418610,1.934302,19441.725057,4.527199e-02,0.074769,23.369120,2.045145,1.857942,0.054222,1.867810,1.070415,...,0.020510,3.095795,0.001957,2.514198e+06,4.187711,0.020254,0.069522,0.000129,0.093145,1.0
38581294,2.838803,78431.581990,6.626930e-02,0.079135,51.577812,3.572165,2.755322,0.071705,2.773258,0.989683,...,0.155149,4.271371,0.003204,2.847141e+06,3.673864,0.005602,0.667983,0.000255,0.144014,1.0
47730588,1.913005,21963.054677,5.460113e-02,0.047842,21.389449,2.189460,1.844602,0.068655,1.878945,0.932321,...,0.036400,3.559080,0.003893,1.294095e+06,8.059244,0.004167,0.933599,0.001423,0.053240,2.0
49696709,2.981339,81806.067277,1.434771e-02,0.111454,29.867949,3.472855,2.858551,0.082533,2.870545,0.975520,...,0.065477,4.001823,0.002302,3.862245e+06,2.817298,0.007240,0.583065,0.000092,0.189203,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56124435,2.397334,79676.170423,2.077586e-01,0.094314,21.825817,2.655513,2.288176,0.080719,2.314499,1.040494,...,0.060496,4.704448,0.002901,3.836082e+06,17.611352,0.001549,1.395922,0.001495,0.039096,1.0
50438996,2.471583,16799.993618,2.513645e-01,0.118093,22.979982,2.718759,2.339055,0.110733,2.382084,0.961707,...,0.057959,3.456565,0.004921,2.752990e+05,8.664636,0.003766,2.898350,0.008258,0.079879,0.0
54990434,2.204391,12391.249792,3.038270e-01,0.060698,32.538823,2.496660,2.133809,0.058552,2.156958,0.992981,...,0.107283,2.977217,0.005895,3.633319e+05,3.167530,0.010460,0.948817,0.001024,0.123691,2.0


In [7]:
feature = pre.drop(columns='encoded_PDL1')
target = pre['encoded_PDL1']
list_numerical = feature.columns.to_list()
list_numerical

['pre_exponential_firstorder_90Percentile',
 'pre_exponential_firstorder_Energy',
 'pre_exponential_firstorder_Entropy',
 'pre_exponential_firstorder_InterquartileRange',
 'pre_exponential_firstorder_Kurtosis',
 'pre_exponential_firstorder_Maximum',
 'pre_exponential_firstorder_Mean',
 'pre_exponential_firstorder_MeanAbsoluteDeviation',
 'pre_exponential_firstorder_Median',
 'pre_exponential_firstorder_Minimum',
 'pre_exponential_firstorder_Range',
 'pre_exponential_firstorder_RobustMeanAbsoluteDeviation',
 'pre_exponential_firstorder_RootMeanSquared',
 'pre_exponential_firstorder_Skewness',
 'pre_exponential_firstorder_TotalEnergy',
 'pre_exponential_firstorder_Uniformity',
 'pre_exponential_firstorder_Variance',
 'pre_exponential_glcm_Autocorrelation',
 'pre_exponential_glcm_ClusterProminence',
 'pre_exponential_glcm_ClusterShade',
 'pre_exponential_glcm_ClusterTendency',
 'pre_exponential_glcm_Contrast',
 'pre_exponential_glcm_Correlation',
 'pre_exponential_glcm_DifferenceAverage',

In [8]:
feature

,pre_exponential_firstorder_90Percentile,pre_exponential_firstorder_Energy,pre_exponential_firstorder_Entropy,pre_exponential_firstorder_InterquartileRange,pre_exponential_firstorder_Kurtosis,pre_exponential_firstorder_Maximum,pre_exponential_firstorder_Mean,pre_exponential_firstorder_MeanAbsoluteDeviation,pre_exponential_firstorder_Median,pre_exponential_firstorder_Minimum,...,pre_wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,pre_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,pre_wavelet-LLL_glszm_ZoneEntropy,pre_wavelet-LLL_glszm_ZonePercentage,pre_wavelet-LLL_glszm_ZoneVariance,pre_wavelet-LLL_ngtdm_Busyness,pre_wavelet-LLL_ngtdm_Coarseness,pre_wavelet-LLL_ngtdm_Complexity,pre_wavelet-LLL_ngtdm_Contrast,pre_wavelet-LLL_ngtdm_Strength
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,1.488204,33747.767654,-3.203427e-16,0.029460,16.777797,1.663273,1.438424,0.050884,1.453668,0.849846,...,2.612107,0.010728,2.807355,0.000432,2.843862e+07,9.302526,0.006286,0.172151,0.000534,0.029250
48418610,1.934302,19441.725057,4.527199e-02,0.074769,23.369120,2.045145,1.857942,0.054222,1.867810,1.070415,...,0.300325,0.020510,3.095795,0.001957,2.514198e+06,4.187711,0.020254,0.069522,0.000129,0.093145
38581294,2.838803,78431.581990,6.626930e-02,0.079135,51.577812,3.572165,2.755322,0.071705,2.773258,0.989683,...,4.913526,0.155149,4.271371,0.003204,2.847141e+06,3.673864,0.005602,0.667983,0.000255,0.144014
47730588,1.913005,21963.054677,5.460113e-02,0.047842,21.389449,2.189460,1.844602,0.068655,1.878945,0.932321,...,7.473821,0.036400,3.559080,0.003893,1.294095e+06,8.059244,0.004167,0.933599,0.001423,0.053240
49696709,2.981339,81806.067277,1.434771e-02,0.111454,29.867949,3.472855,2.858551,0.082533,2.870545,0.975520,...,5.434147,0.065477,4.001823,0.002302,3.862245e+06,2.817298,0.007240,0.583065,0.000092,0.189203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56124435,2.397334,79676.170423,2.077586e-01,0.094314,21.825817,2.655513,2.288176,0.080719,2.314499,1.040494,...,4.800308,0.060496,4.704448,0.002901,3.836082e+06,17.611352,0.001549,1.395922,0.001495,0.039096
50438996,2.471583,16799.993618,2.513645e-01,0.118093,22.979982,2.718759,2.339055,0.110733,2.382084,0.961707,...,2.776515,0.057959,3.456565,0.004921,2.752990e+05,8.664636,0.003766,2.898350,0.008258,0.079879
54990434,2.204391,12391.249792,3.038270e-01,0.060698,32.538823,2.496660,2.133809,0.058552,2.156958,0.992981,...,2.060174,0.107283,2.977217,0.005895,3.633319e+05,3.167530,0.010460,0.948817,0.001024,0.123691


In [9]:
# def custom_zscore(column):
#     if column.std() == 0:
#         return (column - column.mean())  # For columns with zero variance, mean-centered values
#     else:
#         return (column - column.mean()) / column.std()

# # Apply custom_zscore to each column
# feature = feature.apply(custom_zscore)
# feature

In [10]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import Lasso, LassoCV
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

In [11]:
# lasso 를 위한 데이터 나누기 
X_train, X_test, y_train, y_test = train_test_split(feature, target, shuffle=True, test_size = 0.2, stratify=target, random_state=2023)

In [12]:
X_train

,pre_exponential_firstorder_90Percentile,pre_exponential_firstorder_Energy,pre_exponential_firstorder_Entropy,pre_exponential_firstorder_InterquartileRange,pre_exponential_firstorder_Kurtosis,pre_exponential_firstorder_Maximum,pre_exponential_firstorder_Mean,pre_exponential_firstorder_MeanAbsoluteDeviation,pre_exponential_firstorder_Median,pre_exponential_firstorder_Minimum,...,pre_wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,pre_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,pre_wavelet-LLL_glszm_ZoneEntropy,pre_wavelet-LLL_glszm_ZonePercentage,pre_wavelet-LLL_glszm_ZoneVariance,pre_wavelet-LLL_ngtdm_Busyness,pre_wavelet-LLL_ngtdm_Coarseness,pre_wavelet-LLL_ngtdm_Complexity,pre_wavelet-LLL_ngtdm_Contrast,pre_wavelet-LLL_ngtdm_Strength
pt_no,,,,,,,,,,,,,,,,,,,,,
54502130,2.466329,22524.587614,0.163975,0.059693,39.777076,2.699933,2.387414,0.062297,2.411276,1.012191,...,9.046732,0.026684,3.308695,0.005077,6.875654e+05,2.344466,0.013687,0.443903,0.000246,0.176229
43635890,2.436003,20016.887487,0.981413,0.458551,4.261806,2.515758,2.062483,0.270849,2.040290,0.907718,...,5.333161,0.015270,3.702820,0.003501,4.144474e+05,5.404833,0.007589,1.022379,0.006840,0.087973
44284334,2.544196,15154.384426,0.303675,0.114769,16.882651,2.722580,2.401268,0.125513,2.457691,0.984475,...,7.144420,0.011612,3.240224,0.005758,2.992336e+05,5.642488,0.006126,1.954694,0.003844,0.082310
50481279,2.636056,10190.324338,0.050772,0.084444,22.123178,2.969757,2.540611,0.067626,2.553719,1.434311,...,3.027783,0.189236,2.725481,0.005711,1.171943e+05,11.320079,0.005042,2.283562,0.010853,0.053897
49645129,2.117603,6776.218303,0.947970,0.122444,13.524147,2.263390,1.991028,0.104866,2.033667,0.964762,...,1.360623,0.085090,2.641604,0.005303,1.121409e+05,30.185869,0.005037,0.916566,0.025553,0.017418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47047248,3.112265,40528.852390,0.790495,0.688836,3.991168,4.447892,2.449423,0.402541,2.340935,1.673503,...,2.061892,0.031386,4.070656,0.003236,3.169239e+05,37.114070,0.002752,2.003494,0.014831,0.026385
55202394,2.464039,5471.859145,0.455360,0.161416,10.612075,3.051057,2.288033,0.154632,2.353156,0.933304,...,5.676748,0.097624,3.577820,0.015474,4.563761e+04,1.611170,0.013658,2.770416,0.002528,0.353236
47181472,2.156098,181422.512370,0.724754,0.094567,29.581814,2.497098,2.060762,0.069891,2.081184,0.859864,...,5.441477,0.023056,4.235491,0.001173,2.318269e+07,28.936945,0.000779,1.216805,0.000851,0.020208


In [13]:
X_test

,pre_exponential_firstorder_90Percentile,pre_exponential_firstorder_Energy,pre_exponential_firstorder_Entropy,pre_exponential_firstorder_InterquartileRange,pre_exponential_firstorder_Kurtosis,pre_exponential_firstorder_Maximum,pre_exponential_firstorder_Mean,pre_exponential_firstorder_MeanAbsoluteDeviation,pre_exponential_firstorder_Median,pre_exponential_firstorder_Minimum,...,pre_wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,pre_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,pre_wavelet-LLL_glszm_ZoneEntropy,pre_wavelet-LLL_glszm_ZonePercentage,pre_wavelet-LLL_glszm_ZoneVariance,pre_wavelet-LLL_ngtdm_Busyness,pre_wavelet-LLL_ngtdm_Coarseness,pre_wavelet-LLL_ngtdm_Complexity,pre_wavelet-LLL_ngtdm_Contrast,pre_wavelet-LLL_ngtdm_Strength
pt_no,,,,,,,,,,,,,,,,,,,,,
56241796,2.205686,44192.176786,2.439644e-01,0.060380,51.854428,2.291859,2.139294,0.050816,2.156819,0.908566,...,4.930739,0.043896,3.773557,0.003113,2.743468e+06,4.600876,0.005585,0.627700,0.000279,0.140036
48418610,1.934302,19441.725057,4.527199e-02,0.074769,23.369120,2.045145,1.857942,0.054222,1.867810,1.070415,...,0.300325,0.020510,3.095795,0.001957,2.514198e+06,4.187711,0.020254,0.069522,0.000129,0.093145
42120720,2.415367,15653.568604,4.511923e-01,0.145168,11.376814,2.808382,2.238054,0.178562,2.334932,0.947927,...,8.099204,0.031398,3.767183,0.008469,1.373551e+05,6.915411,0.003425,5.216459,0.014164,0.107463
47563562,2.255204,22117.557467,3.957427e-01,0.121645,5.837155,2.620923,2.125385,0.073743,2.118263,1.353551,...,0.897641,0.213880,3.085055,0.002661,1.270764e+06,36.341193,0.003952,0.442789,0.004149,0.008852
41480094,1.973194,60767.243887,3.166853e-01,0.140086,11.946075,2.201490,1.806602,0.102695,1.810519,0.826747,...,2.108111,0.027840,3.923856,0.001354,1.109530e+07,15.491502,0.002139,0.625436,0.000871,0.028604
56131318,2.062365,47500.994762,9.834847e-01,0.102299,17.087590,2.338467,1.958564,0.102617,2.013428,0.938527,...,4.665678,0.057831,4.507876,0.004641,2.170356e+06,8.439987,0.002532,1.240894,0.001527,0.063048
42332460,1.888605,29981.931426,-3.203427e-16,0.050872,23.118719,1.996023,1.838097,0.037510,1.847606,1.143954,...,0.807358,0.105607,3.219528,0.001918,4.225750e+06,6.387145,0.013228,0.069089,0.000082,0.062997
52709959,2.788234,28788.754021,1.054304e-01,0.209171,8.876426,3.119040,2.593480,0.143894,2.639361,1.292350,...,2.908012,0.046969,3.459432,0.002585,6.767988e+05,18.055839,0.003376,1.267056,0.007190,0.032917
44758277,2.318496,30405.965886,5.325939e-02,0.108048,9.360290,2.523468,2.227615,0.064231,2.236314,1.282902,...,1.195964,0.156643,3.188722,0.001961,2.175687e+06,32.072008,0.002968,0.452649,0.004686,0.012511


In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train[list_numerical]) 

X_train[list_numerical] = scaler.transform(X_train[list_numerical])

X_test[list_numerical] = scaler.transform(X_test[list_numerical])

In [15]:
X_train

,pre_exponential_firstorder_90Percentile,pre_exponential_firstorder_Energy,pre_exponential_firstorder_Entropy,pre_exponential_firstorder_InterquartileRange,pre_exponential_firstorder_Kurtosis,pre_exponential_firstorder_Maximum,pre_exponential_firstorder_Mean,pre_exponential_firstorder_MeanAbsoluteDeviation,pre_exponential_firstorder_Median,pre_exponential_firstorder_Minimum,...,pre_wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,pre_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,pre_wavelet-LLL_glszm_ZoneEntropy,pre_wavelet-LLL_glszm_ZonePercentage,pre_wavelet-LLL_glszm_ZoneVariance,pre_wavelet-LLL_ngtdm_Busyness,pre_wavelet-LLL_ngtdm_Coarseness,pre_wavelet-LLL_ngtdm_Complexity,pre_wavelet-LLL_ngtdm_Contrast,pre_wavelet-LLL_ngtdm_Strength
pt_no,,,,,,,,,,,,,,,,,,,,,
54502130,0.650562,-0.408103,-0.560178,-0.450228,0.041651,-0.138752,0.917099,-0.489808,0.880960,-0.416562,...,1.966973,-0.588468,-0.215857,0.228556,-0.466435,-0.177955,-0.112509,-0.550393,-0.647345,0.377820
43635890,0.556607,-0.481936,1.982046,2.442111,-0.310532,-0.255044,-0.235920,1.955640,-0.417456,-0.787047,...,0.579026,-0.687924,0.236009,-0.200817,-0.502686,-0.166103,-0.112509,-0.267945,0.142926,-0.100852
44284334,0.891813,-0.625101,-0.125713,-0.050848,-0.185379,-0.124453,0.966259,0.251453,1.043408,-0.514848,...,1.255984,-0.719808,-0.294360,0.413976,-0.517979,-0.165183,-0.112509,0.187272,-0.216130,-0.131566
50481279,1.176413,-0.771255,-0.912239,-0.270750,-0.133412,0.031619,1.460716,-0.427316,1.379494,1.080371,...,-0.282608,0.828049,-0.884517,0.401038,-0.542141,-0.143196,-0.112509,0.347846,0.623957,-0.285669
49645129,-0.429866,-0.871775,1.878038,0.004814,-0.218683,-0.414393,-0.489478,0.009356,-0.440637,-0.584755,...,-0.905709,-0.079500,-0.980682,0.290127,-0.542812,-0.070137,-0.112509,-0.319609,2.385785,-0.483523
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47047248,2.651811,0.121988,1.388293,4.112032,-0.313215,0.964939,1.137138,3.499823,0.634773,1.928599,...,-0.643610,-0.547492,0.657737,-0.272958,-0.515631,-0.043307,-0.112509,0.211099,1.100723,-0.434889
55202394,0.643469,-0.910179,0.346025,0.287417,-0.247560,0.082953,0.564445,0.592897,0.677543,-0.696310,...,0.707442,0.029720,0.092696,3.060406,-0.551639,-0.180795,-0.112509,0.585560,-0.373844,1.337851
47181472,-0.310599,4.270254,1.183840,-0.197340,-0.059449,-0.266826,-0.242025,-0.400755,-0.274332,-0.956747,...,0.619510,-0.620078,0.846721,-0.834824,2.519391,-0.074973,-0.112509,-0.173013,-0.574835,-0.468393


# change

In [16]:
n_alphas = 100
alphas = np.logspace(-4, -1.8, n_alphas)
print(alphas)
lasso_cv = LassoCV(alphas = alphas, random_state=2023, max_iter=1000000, cv=KFold(n_splits=5), verbose=True, n_jobs=-1).fit(X_train, y_train)

[0.0001     0.00010525 0.00011078 0.00011659 0.00012271 0.00012915
 0.00013594 0.00014307 0.00015058 0.00015849 0.00016681 0.00017557
 0.00018478 0.00019449 0.0002047  0.00021544 0.00022675 0.00023866
 0.00025119 0.00026438 0.00027826 0.00029286 0.00030824 0.00032442
 0.00034145 0.00035938 0.00037825 0.00039811 0.00041901 0.00044101
 0.00046416 0.00048853 0.00051418 0.00054117 0.00056958 0.00059948
 0.00063096 0.00066408 0.00069895 0.00073564 0.00077426 0.00081491
 0.0008577  0.00090273 0.00095012 0.001      0.0010525  0.00110776
 0.00116591 0.00122713 0.00129155 0.00135936 0.00143072 0.00150584
 0.00158489 0.0016681  0.00175568 0.00184785 0.00194486 0.00204697
 0.00215443 0.00226754 0.00238659 0.00251189 0.00264376 0.00278256
 0.00292864 0.0030824  0.00324423 0.00341455 0.00359381 0.00378249
 0.00398107 0.00419008 0.00441006 0.00464159 0.00488527 0.00514175
 0.0054117  0.00569581 0.00599484 0.00630957 0.00664083 0.00698947
 0.00735642 0.00774264 0.00814913 0.00857696 0.00902725 0.0095

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.6s finished


In [17]:
print('R squared training set', round(lasso_cv.score(X_train, y_train)*100, 2))
print('R squared test set', round(lasso_cv.score(X_test, y_test)*100, 2))

R squared training set 94.9
R squared test set 35.93


In [18]:
best_alpha = lasso_cv.alpha_
print("Best alpha parameter:", best_alpha)

Best alpha parameter: 0.015848931924611134


In [19]:
feature_list = lasso_cv.feature_names_in_[lasso_cv.coef_ != 0]
print(len(feature_list), feature_list)

72 ['pre_exponential_glcm_Imc1' 'pre_exponential_gldm_DependenceVariance'
 'pre_exponential_glrlm_ShortRunLowGrayLevelEmphasis'
 'pre_exponential_glszm_SizeZoneNonUniformity'
 'pre_exponential_glszm_ZoneVariance' 'pre_gradient_ngtdm_Strength'
 'pre_lbp-2D_glcm_JointAverage'
 'pre_lbp-2D_gldm_LargeDependenceLowGrayLevelEmphasis'
 'pre_lbp-2D_glszm_GrayLevelNonUniformityNormalized'
 'pre_log-sigma-0-5-mm-3D_glrlm_GrayLevelNonUniformityNormalized'
 'pre_log-sigma-0-5-mm-3D_glszm_GrayLevelNonUniformityNormalized'
 'pre_log-sigma-0-5-mm-3D_glszm_SmallAreaLowGrayLevelEmphasis'
 'pre_log-sigma-0-5-mm-3D_glszm_ZoneEntropy'
 'pre_log-sigma-1-0-mm-3D_glszm_SizeZoneNonUniformity'
 'pre_log-sigma-1-0-mm-3D_glszm_SmallAreaLowGrayLevelEmphasis'
 'pre_log-sigma-1-5-mm-3D_gldm_SmallDependenceLowGrayLevelEmphasis'
 'pre_log-sigma-1-5-mm-3D_glszm_GrayLevelNonUniformityNormalized'
 'pre_log-sigma-1-5-mm-3D_glszm_ZoneEntropy'
 'pre_log-sigma-2-0-mm-3D_glcm_Imc1'
 'pre_log-sigma-2-0-mm-3D_glszm_SizeZoneNon

In [20]:
lasso_data = feature[feature_list]
lasso_data

,pre_exponential_glcm_Imc1,pre_exponential_gldm_DependenceVariance,pre_exponential_glrlm_ShortRunLowGrayLevelEmphasis,pre_exponential_glszm_SizeZoneNonUniformity,pre_exponential_glszm_ZoneVariance,pre_gradient_ngtdm_Strength,pre_lbp-2D_glcm_JointAverage,pre_lbp-2D_gldm_LargeDependenceLowGrayLevelEmphasis,pre_lbp-2D_glszm_GrayLevelNonUniformityNormalized,pre_log-sigma-0-5-mm-3D_glrlm_GrayLevelNonUniformityNormalized,...,pre_wavelet-HLL_ngtdm_Strength,pre_wavelet-LHL_glcm_ClusterShade,pre_wavelet-LHL_glcm_MaximumProbability,pre_wavelet-LHL_glszm_ZoneEntropy,pre_wavelet-LLH_firstorder_90Percentile,pre_wavelet-LLH_firstorder_Uniformity,pre_wavelet-LLL_gldm_LargeDependenceLowGrayLevelEmphasis,pre_wavelet-LLL_glrlm_LowGrayLevelRunEmphasis,pre_wavelet-LLL_glrlm_ShortRunHighGrayLevelEmphasis,pre_wavelet-LLL_glszm_SizeZoneNonUniformity
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,0.000000,21.969881,0.185579,1.000000,0.000000e+00,0.000000,3.280068,10.747121,0.321866,0.500066,...,0.000288,-0.002872,0.283745,2.725481,0.030997,0.500168,71.057968,0.207526,1.941581,1.000000
48418610,-0.029668,26.654605,0.129953,2.272727,2.582677e+06,0.000000,3.069902,11.439973,0.260536,0.500553,...,0.000787,0.055007,0.316773,2.000000,0.070327,0.502805,60.740957,0.129203,1.314703,1.545455
38581294,-0.214270,25.517720,0.091120,2.714286,1.275626e+07,0.000000,3.302955,8.529810,0.292130,0.500437,...,0.002592,0.026725,0.298173,2.281036,0.078478,0.500039,22.983520,0.056000,5.912600,6.030303
47730588,-0.108681,30.046907,0.201122,3.200000,3.659475e+06,0.000000,3.251733,10.370738,0.345481,0.500259,...,0.004345,0.036818,0.308398,2.406666,0.114351,0.501203,31.925714,0.097787,4.373449,5.640000
49696709,-0.212127,18.457690,0.030920,1.500000,1.865514e+07,0.000000,3.153295,10.165948,0.256496,0.500036,...,0.002923,0.063339,0.326645,2.521641,0.066627,0.500716,24.051753,0.046530,4.806990,3.608696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56124435,-0.227156,26.535603,0.135294,2.600000,1.019091e+07,0.000000,3.189922,10.447437,0.257840,0.500108,...,0.001187,0.042890,0.308225,2.777363,0.066353,0.501846,21.902516,0.055013,5.949518,6.727273
50438996,-0.378442,31.549635,0.105288,1.333333,1.165044e+06,0.000000,3.134253,10.718051,0.220473,0.501657,...,0.010186,0.037129,0.320285,1.584963,0.098511,0.500005,18.200812,0.070279,8.714608,2.333333
54990434,-0.115942,40.066341,0.216651,2.714286,4.334870e+05,0.000000,3.350842,7.638302,0.322070,0.500320,...,0.010676,0.015875,0.288662,1.921928,0.129075,0.500413,30.523804,0.086007,3.929203,3.500000


In [21]:
final_data = pd.concat([lasso_data, target], axis=1)
final_data

,pre_exponential_glcm_Imc1,pre_exponential_gldm_DependenceVariance,pre_exponential_glrlm_ShortRunLowGrayLevelEmphasis,pre_exponential_glszm_SizeZoneNonUniformity,pre_exponential_glszm_ZoneVariance,pre_gradient_ngtdm_Strength,pre_lbp-2D_glcm_JointAverage,pre_lbp-2D_gldm_LargeDependenceLowGrayLevelEmphasis,pre_lbp-2D_glszm_GrayLevelNonUniformityNormalized,pre_log-sigma-0-5-mm-3D_glrlm_GrayLevelNonUniformityNormalized,...,pre_wavelet-LHL_glcm_ClusterShade,pre_wavelet-LHL_glcm_MaximumProbability,pre_wavelet-LHL_glszm_ZoneEntropy,pre_wavelet-LLH_firstorder_90Percentile,pre_wavelet-LLH_firstorder_Uniformity,pre_wavelet-LLL_gldm_LargeDependenceLowGrayLevelEmphasis,pre_wavelet-LLL_glrlm_LowGrayLevelRunEmphasis,pre_wavelet-LLL_glrlm_ShortRunHighGrayLevelEmphasis,pre_wavelet-LLL_glszm_SizeZoneNonUniformity,encoded_PDL1
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,0.000000,21.969881,0.185579,1.000000,0.000000e+00,0.000000,3.280068,10.747121,0.321866,0.500066,...,-0.002872,0.283745,2.725481,0.030997,0.500168,71.057968,0.207526,1.941581,1.000000,1.0
48418610,-0.029668,26.654605,0.129953,2.272727,2.582677e+06,0.000000,3.069902,11.439973,0.260536,0.500553,...,0.055007,0.316773,2.000000,0.070327,0.502805,60.740957,0.129203,1.314703,1.545455,1.0
38581294,-0.214270,25.517720,0.091120,2.714286,1.275626e+07,0.000000,3.302955,8.529810,0.292130,0.500437,...,0.026725,0.298173,2.281036,0.078478,0.500039,22.983520,0.056000,5.912600,6.030303,1.0
47730588,-0.108681,30.046907,0.201122,3.200000,3.659475e+06,0.000000,3.251733,10.370738,0.345481,0.500259,...,0.036818,0.308398,2.406666,0.114351,0.501203,31.925714,0.097787,4.373449,5.640000,2.0
49696709,-0.212127,18.457690,0.030920,1.500000,1.865514e+07,0.000000,3.153295,10.165948,0.256496,0.500036,...,0.063339,0.326645,2.521641,0.066627,0.500716,24.051753,0.046530,4.806990,3.608696,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56124435,-0.227156,26.535603,0.135294,2.600000,1.019091e+07,0.000000,3.189922,10.447437,0.257840,0.500108,...,0.042890,0.308225,2.777363,0.066353,0.501846,21.902516,0.055013,5.949518,6.727273,1.0
50438996,-0.378442,31.549635,0.105288,1.333333,1.165044e+06,0.000000,3.134253,10.718051,0.220473,0.501657,...,0.037129,0.320285,1.584963,0.098511,0.500005,18.200812,0.070279,8.714608,2.333333,0.0
54990434,-0.115942,40.066341,0.216651,2.714286,4.334870e+05,0.000000,3.350842,7.638302,0.322070,0.500320,...,0.015875,0.288662,1.921928,0.129075,0.500413,30.523804,0.086007,3.929203,3.500000,2.0


In [22]:
# clinical_train = pd.read_csv("/media/yonsei/Mike/0818_101명_기준/2FDATA_clinical_train.csv", index_col=0) 
# clinical_test = pd.read_csv("/media/yonsei/Mike/0818_101명_기준/2FDATA_clinical_test.csv", index_col=0) 
clinical_test = pd.read_csv (r"C:\Users\user\Desktop\vsc\Severance_Hospital\12.19_정현도_alpha_feature\pre\2FDATA_clinical_test.csv", index_col=0)
clinical_train = pd.read_csv (r"C:\Users\user\Desktop\vsc\Severance_Hospital\12.19_정현도_alpha_feature\pre\2FDATA_clinical_train.csv" , index_col=0)

clinical_train.drop(columns=['TD', 'RT.tech_IMRT', 'location'], inplace=True)
clinical_test.drop(columns=['TD', 'RT.tech_IMRT', 'location'], inplace=True)
clinical_train

,encoded_PDL1
pt_no,
51127213,1.0
38924846,0.0
56241796,1.0
40815882,1.0
33908849,0.0
...,...
47181472,2.0
49087653,1.0
49588132,0.0


In [23]:
rad_features_list = final_data.columns.to_list()
rad_features_list.remove('encoded_PDL1')
rad_features_list

['pre_exponential_glcm_Imc1',
 'pre_exponential_gldm_DependenceVariance',
 'pre_exponential_glrlm_ShortRunLowGrayLevelEmphasis',
 'pre_exponential_glszm_SizeZoneNonUniformity',
 'pre_exponential_glszm_ZoneVariance',
 'pre_gradient_ngtdm_Strength',
 'pre_lbp-2D_glcm_JointAverage',
 'pre_lbp-2D_gldm_LargeDependenceLowGrayLevelEmphasis',
 'pre_lbp-2D_glszm_GrayLevelNonUniformityNormalized',
 'pre_log-sigma-0-5-mm-3D_glrlm_GrayLevelNonUniformityNormalized',
 'pre_log-sigma-0-5-mm-3D_glszm_GrayLevelNonUniformityNormalized',
 'pre_log-sigma-0-5-mm-3D_glszm_SmallAreaLowGrayLevelEmphasis',
 'pre_log-sigma-0-5-mm-3D_glszm_ZoneEntropy',
 'pre_log-sigma-1-0-mm-3D_glszm_SizeZoneNonUniformity',
 'pre_log-sigma-1-0-mm-3D_glszm_SmallAreaLowGrayLevelEmphasis',
 'pre_log-sigma-1-5-mm-3D_gldm_SmallDependenceLowGrayLevelEmphasis',
 'pre_log-sigma-1-5-mm-3D_glszm_GrayLevelNonUniformityNormalized',
 'pre_log-sigma-1-5-mm-3D_glszm_ZoneEntropy',
 'pre_log-sigma-2-0-mm-3D_glcm_Imc1',
 'pre_log-sigma-2-0-mm-3D

In [24]:
clinical_train[rad_features_list] = final_data[rad_features_list]
clinical_test[rad_features_list] = final_data[rad_features_list]
clinical_train

,encoded_PDL1,pre_exponential_glcm_Imc1,pre_exponential_gldm_DependenceVariance,pre_exponential_glrlm_ShortRunLowGrayLevelEmphasis,pre_exponential_glszm_SizeZoneNonUniformity,pre_exponential_glszm_ZoneVariance,pre_gradient_ngtdm_Strength,pre_lbp-2D_glcm_JointAverage,pre_lbp-2D_gldm_LargeDependenceLowGrayLevelEmphasis,pre_lbp-2D_glszm_GrayLevelNonUniformityNormalized,...,pre_wavelet-HLL_ngtdm_Strength,pre_wavelet-LHL_glcm_ClusterShade,pre_wavelet-LHL_glcm_MaximumProbability,pre_wavelet-LHL_glszm_ZoneEntropy,pre_wavelet-LLH_firstorder_90Percentile,pre_wavelet-LLH_firstorder_Uniformity,pre_wavelet-LLL_gldm_LargeDependenceLowGrayLevelEmphasis,pre_wavelet-LLL_glrlm_LowGrayLevelRunEmphasis,pre_wavelet-LLL_glrlm_ShortRunHighGrayLevelEmphasis,pre_wavelet-LLL_glszm_SizeZoneNonUniformity
pt_no,,,,,,,,,,,,,,,,,,,,,
51127213,1.0,-0.423950,37.605991,0.175694,1.000000,7.851155e+05,0.000000,2.994978,15.686878,0.261224,...,0.001506,0.108622,0.374988,3.734522,0.222959,0.500138,19.971682,0.082225,8.034524,3.428571
38924846,0.0,-0.220052,29.920214,0.111101,2.090909,1.019429e+06,0.000000,3.194165,9.786681,0.273139,...,0.016104,0.101090,0.361245,1.584963,0.089724,0.513613,29.849558,0.070572,5.278268,2.619048
56241796,1.0,-0.160116,33.720570,0.193787,11.857143,1.983099e+06,0.000000,3.277453,8.493429,0.326882,...,0.001811,0.045127,0.307869,2.732915,0.061699,0.502268,22.518602,0.049361,5.625945,5.733333
40815882,1.0,-0.088454,29.770789,0.108237,2.666667,1.732604e+06,0.000000,3.213287,11.101707,0.270155,...,0.003703,0.041194,0.306524,2.321928,0.050280,0.500491,62.079343,0.146348,1.852701,1.600000
33908849,0.0,-0.510434,31.268762,0.150639,2.571429,7.123570e+06,0.125267,3.101316,13.814964,0.238403,...,0.002829,0.002758,0.305031,3.001609,0.181989,0.500385,29.773179,0.113539,10.236524,6.642857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47181472,2.0,-0.498196,33.024084,0.221229,28.758242,1.321480e+07,0.000000,3.278112,9.803558,0.282012,...,0.000615,0.017900,0.293185,3.072169,0.055339,0.500352,24.826203,0.053398,5.454939,6.040000
49087653,1.0,0.000000,25.452702,0.182363,1.000000,0.000000e+00,0.000000,3.192794,9.412404,0.245013,...,0.003664,0.053060,0.324866,2.405639,0.100649,0.501508,18.748511,0.074835,7.475374,4.785714
49588132,0.0,-0.100259,18.614016,0.061034,6.120000,2.018546e+07,0.000000,3.285952,9.364063,0.332441,...,0.001200,0.025991,0.297384,2.947703,0.044044,0.500561,66.996446,0.107427,3.143245,5.458333


In [25]:
clinical_test

,encoded_PDL1,pre_exponential_glcm_Imc1,pre_exponential_gldm_DependenceVariance,pre_exponential_glrlm_ShortRunLowGrayLevelEmphasis,pre_exponential_glszm_SizeZoneNonUniformity,pre_exponential_glszm_ZoneVariance,pre_gradient_ngtdm_Strength,pre_lbp-2D_glcm_JointAverage,pre_lbp-2D_gldm_LargeDependenceLowGrayLevelEmphasis,pre_lbp-2D_glszm_GrayLevelNonUniformityNormalized,...,pre_wavelet-HLL_ngtdm_Strength,pre_wavelet-LHL_glcm_ClusterShade,pre_wavelet-LHL_glcm_MaximumProbability,pre_wavelet-LHL_glszm_ZoneEntropy,pre_wavelet-LLH_firstorder_90Percentile,pre_wavelet-LLH_firstorder_Uniformity,pre_wavelet-LLL_gldm_LargeDependenceLowGrayLevelEmphasis,pre_wavelet-LLL_glrlm_LowGrayLevelRunEmphasis,pre_wavelet-LLL_glrlm_ShortRunHighGrayLevelEmphasis,pre_wavelet-LLL_glszm_SizeZoneNonUniformity
pt_no,,,,,,,,,,,,,,,,,,,,,
49645129,1.0,-0.127345,42.975696,0.382098,1.222222,1.314251e+05,0.000000,2.892661,11.729083,0.284560,...,0.015469,0.058546,0.329828,2.251629,0.099562,0.507584,46.510361,0.209021,3.090107,1.666667
47451087,2.0,0.000000,19.740838,0.148120,1.000000,0.000000e+00,0.000000,3.198459,10.190806,0.233448,...,0.001209,0.031003,0.306230,2.852217,0.081357,0.501762,23.140914,0.056429,5.772072,10.785714
50247936,2.0,-0.325617,46.208395,0.292406,3.400000,8.440682e+05,0.000000,3.213115,8.843917,0.228772,...,0.000989,0.067579,0.329978,2.664498,0.140668,0.500086,61.315655,0.187722,2.310674,1.000000
50270330,0.0,-0.523732,28.771533,0.101676,1.333333,5.476652e+06,0.000000,3.156903,12.003551,0.261555,...,0.004144,0.014571,0.289981,2.000000,0.096097,0.500653,32.604691,0.115529,6.674605,2.809524
46695156,0.0,-0.192775,33.458494,0.207785,3.000000,5.299158e+05,0.103788,3.068870,10.113091,0.249317,...,0.091553,-0.086965,0.298219,3.251629,0.152442,0.496818,26.392219,0.067295,9.540807,3.750000
42522605,1.0,-0.265027,41.687201,0.243492,5.461538,1.696123e+06,0.000000,3.259428,10.819780,0.295525,...,0.002134,0.044603,0.313190,2.807355,0.079699,0.500796,56.188448,0.147931,3.958742,4.117647
43986877,0.0,-0.176476,25.167020,0.120215,1.000000,4.724102e+06,0.000000,3.227484,9.684084,0.335804,...,0.003542,0.058791,0.320929,1.500000,0.066056,0.500730,31.114703,0.094557,5.403107,1.500000
52779802,1.0,-0.232853,41.039088,0.251154,1.400000,6.828446e+05,0.000000,3.113798,10.957453,0.236570,...,0.001628,0.025292,0.299393,2.321928,0.084408,0.501006,203.307135,0.610176,0.825333,1.000000
43876141,1.0,-0.466160,29.778608,0.102561,4.066667,5.011297e+06,0.000000,3.274063,9.705684,0.367102,...,0.003034,0.007357,0.285453,2.128085,0.059909,0.500130,20.958650,0.065719,6.623908,1.875000


In [26]:
train_data = clinical_train
test_data = clinical_test

In [27]:
# train_data, test_data = train_test_split(final_data, test_size=0.2, shuffle=True, stratify=target, random_state=2021)

In [28]:
train_data

,encoded_PDL1,pre_exponential_glcm_Imc1,pre_exponential_gldm_DependenceVariance,pre_exponential_glrlm_ShortRunLowGrayLevelEmphasis,pre_exponential_glszm_SizeZoneNonUniformity,pre_exponential_glszm_ZoneVariance,pre_gradient_ngtdm_Strength,pre_lbp-2D_glcm_JointAverage,pre_lbp-2D_gldm_LargeDependenceLowGrayLevelEmphasis,pre_lbp-2D_glszm_GrayLevelNonUniformityNormalized,...,pre_wavelet-HLL_ngtdm_Strength,pre_wavelet-LHL_glcm_ClusterShade,pre_wavelet-LHL_glcm_MaximumProbability,pre_wavelet-LHL_glszm_ZoneEntropy,pre_wavelet-LLH_firstorder_90Percentile,pre_wavelet-LLH_firstorder_Uniformity,pre_wavelet-LLL_gldm_LargeDependenceLowGrayLevelEmphasis,pre_wavelet-LLL_glrlm_LowGrayLevelRunEmphasis,pre_wavelet-LLL_glrlm_ShortRunHighGrayLevelEmphasis,pre_wavelet-LLL_glszm_SizeZoneNonUniformity
pt_no,,,,,,,,,,,,,,,,,,,,,
51127213,1.0,-0.423950,37.605991,0.175694,1.000000,7.851155e+05,0.000000,2.994978,15.686878,0.261224,...,0.001506,0.108622,0.374988,3.734522,0.222959,0.500138,19.971682,0.082225,8.034524,3.428571
38924846,0.0,-0.220052,29.920214,0.111101,2.090909,1.019429e+06,0.000000,3.194165,9.786681,0.273139,...,0.016104,0.101090,0.361245,1.584963,0.089724,0.513613,29.849558,0.070572,5.278268,2.619048
56241796,1.0,-0.160116,33.720570,0.193787,11.857143,1.983099e+06,0.000000,3.277453,8.493429,0.326882,...,0.001811,0.045127,0.307869,2.732915,0.061699,0.502268,22.518602,0.049361,5.625945,5.733333
40815882,1.0,-0.088454,29.770789,0.108237,2.666667,1.732604e+06,0.000000,3.213287,11.101707,0.270155,...,0.003703,0.041194,0.306524,2.321928,0.050280,0.500491,62.079343,0.146348,1.852701,1.600000
33908849,0.0,-0.510434,31.268762,0.150639,2.571429,7.123570e+06,0.125267,3.101316,13.814964,0.238403,...,0.002829,0.002758,0.305031,3.001609,0.181989,0.500385,29.773179,0.113539,10.236524,6.642857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47181472,2.0,-0.498196,33.024084,0.221229,28.758242,1.321480e+07,0.000000,3.278112,9.803558,0.282012,...,0.000615,0.017900,0.293185,3.072169,0.055339,0.500352,24.826203,0.053398,5.454939,6.040000
49087653,1.0,0.000000,25.452702,0.182363,1.000000,0.000000e+00,0.000000,3.192794,9.412404,0.245013,...,0.003664,0.053060,0.324866,2.405639,0.100649,0.501508,18.748511,0.074835,7.475374,4.785714
49588132,0.0,-0.100259,18.614016,0.061034,6.120000,2.018546e+07,0.000000,3.285952,9.364063,0.332441,...,0.001200,0.025991,0.297384,2.947703,0.044044,0.500561,66.996446,0.107427,3.143245,5.458333


In [29]:
print('Data for Modeling: ' + str(train_data.shape))
print('Unseen Data For Predictions: ' + str(test_data.shape))

Data for Modeling: (79, 73)
Unseen Data For Predictions: (22, 73)


In [30]:
ypCR_clf = setup(data=train_data,
            test_data=test_data,
            target='encoded_PDL1',
            normalize=True, 
            normalize_method='zscore', 
            data_split_shuffle=True, 
            data_split_stratify='encoded_PDL1',
            fold = 5,
            fold_strategy = 'stratifiedkfold',
            fold_shuffle = True, 
            session_id=3457, 
            use_gpu=True)

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: Intel(R) UHD Graphics 750, Vendor: Intel(R) Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that

,Description,Value
0,Session id,3457
1,Target,encoded_PDL1
2,Target type,Multiclass
3,Original data shape,"(101, 73)"
4,Transformed data shape,"(101, 73)"
5,Transformed train set shape,"(79, 73)"
6,Transformed test set shape,"(22, 73)"
7,Numeric features,72
8,Preprocess,True
9,Imputation type,simple


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: Intel(R) UHD Graphics 750, Vendor: Intel(R) Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that

In [31]:
train = get_config(variable="train_transformed")
train['encoded_PDL1'].value_counts()

1.0    36
0.0    22
2.0    21
Name: encoded_PDL1, dtype: int64

In [32]:
test = get_config(variable="test_transformed")
test['encoded_PDL1'].value_counts()

1.0    10
2.0     6
0.0     6
Name: encoded_PDL1, dtype: int64

# Result

In [33]:
compare_models(n_select=3, sort='AUC')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ada,Ada Boost Classifier,0.5692,0.6798,0.5692,0.5784,0.5302,0.2660,0.3105,0.1420
lr,Logistic Regression,0.4692,0.6534,0.4692,0.4815,0.4674,0.1680,0.1708,0.0380
knn,K Neighbors Classifier,0.4800,0.6318,0.4800,0.5125,0.4625,0.1757,0.1911,0.0600
lightgbm,Light Gradient Boosting Machine,0.4425,0.6150,0.4425,0.4460,0.4310,0.1144,0.1166,0.5660
et,Extra Trees Classifier,0.4167,0.5973,0.4167,0.4926,0.4198,0.0679,0.0731,0.1900
rf,Random Forest Classifier,0.4050,0.5903,0.4050,0.4134,0.3949,0.0455,0.0464,0.2180
gbc,Gradient Boosting Classifier,0.4925,0.5650,0.4925,0.5028,0.4789,0.1916,0.1964,0.6700
lda,Linear Discriminant Analysis,0.3550,0.5602,0.3550,0.4004,0.3408,0.0462,0.0539,0.0360
nb,Naive Bayes,0.3667,0.5537,0.3667,0.3970,0.3662,0.0539,0.0545,0.0340
dummy,Dummy Classifier,0.4558,0.5000,0.4558,0.2084,0.2858,0.0000,0.0000,0.0260


[AdaBoostClassifier(algorithm='SAMME.R', base_estimator='deprecated',
                    estimator=None, learning_rate=1.0, n_estimators=50,
                    random_state=3457),
 LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=1000,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=3457, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                      metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
                      weights='uniform')]

In [34]:
# lightgbm = create_model('lightgbm', fold=5, round=4, cross_validation=True)

In [35]:
# prediction = predict_model(lightgbm, data=test_data)